In [6]:
#Loading Custom Functions
from UTILS.Similarity_search_GUSE_FAISS import *
from UTILS.custom_preprocessing_function import *
from custom_check_intent import *

In [7]:
df = pd.read_csv(r'out.csv')
kpi_df = pd.read_excel(r'kpi_df.xlsx')

In [8]:
#Applying Preprocessing
df=df[~(df.FeedText=='')]
df.dropna(subset=['FeedText'], inplace=True)
df = df.drop_duplicates(subset = ['FeedText'],keep = 'last').reset_index(drop = True)
df['FeedText'] = df['FeedText'].apply(remove_emoji)
df['FeedText'] = df.FeedText.dropna().apply(clean_text).map(lambda x: ' '.join(i for i in x))
df=df[~(df.FeedText=='')]

In [9]:
%%time
result = df.groupby('Id').apply(lambda x: check_intent(x, kpi_df , kpi_type="check_intent"))

CPU times: total: 2h 15min 35s
Wall time: 46min 9s


In [13]:
#Converting the results to dataframe
resultdf= pd.DataFrame(result)
resultdf.reset_index(inplace=True)
resultdf['user_name'] = [i['score'][0][1] if len(i['score'])>0 else 'NA' for i in resultdf[0]]
resultdf['intent'] = [i['score'][0][2] if len(i['score'])>0 else 'NA' for i in resultdf[0]]

In [14]:
resultdf

,Id,0,user_name,intent
0,5b1aa7ee8bb55b300895a774,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
1,5b1aa7ee8bb55b300895a776,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
2,5b1aa7ee8bb55b300895a778,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
3,5b1aa7ee8bb55b300895a77a,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Gian Fortuin,neutral
4,5b1aa7ee8bb55b300895a77c,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Shawn Lemoine,neutral
...,...,...,...,...
134854,5d11b492a3df6323f881ab73,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Marwa Mohamed,neutral
134855,5d11b7e7a3df6323f881ab77,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Oun Sida,neutral
134856,5d11b7e7a3df6323f881ab7b,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Jonathan Reeves,neutral
134857,5d11b7e7a3df6323f881ab7f,"{'kpi_id': 1, 'kpi_name': 'CHECK_INTENT', 'sco...",Michael Dass Sumaylo,neutral


In [12]:
resultdf['intent'].value_counts()

neutral         134456
buyer              250
buyer_seller       105
seller              48
Name: intent, dtype: int64

In [ ]:
resultdf.to_excel('resultdf.xlsx')